In [1]:
import pandas as pd
import warnings
from IPython.utils import io
import sys
import numpy as np
from functools import reduce

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

stars_dir = '~/GitHub/stars-data-builder/'
hos_dir = '~/Desktop/Rush/CMS_HospitalArchives/'

dates_df = pd.DataFrame(columns = ['Measure ID', 'Start Date', 'End Date'])

In [2]:
def curate(df):

    try:
        df = df[df['PROVIDER_ID'] != np.nan]
        df['PROVIDER_ID'] = df['PROVIDER_ID'].values.astype(str)
        
        ids = df['PROVIDER_ID'].tolist()
        ids2 = []
        for i in ids:
            if len(i) < 6:
                i = '0' + i
            ids2.append(i)
        df['PROVIDER_ID'] = ids2
        
    except:
        pass
    
    for c in list(df):    
        try:
            df[c] = df[c].str.replace("\t","")
        except:
            pass

    if 'Unnamed: 0' in list(df):
        df.drop(labels=['Unnamed: 0'], axis=1, inplace=True)
    return df

# Use 2025 SAS input file to get the column labels needed for 2026 prognostication

Remove OP-2 and OP-3B, as these will no longer be reported in Care Compare.   
Will also need to add measures not included in 2025.

In [3]:
sas_input_df = pd.read_sas(stars_dir + '2025/2025-07 Stars Release/alldata_2025jul.sas7bdat', 
                           format = 'sas7bdat', encoding = "utf8")

ls = ['PC_01', 
      'PC_01_DEN',
      'READM_30_HOSP_WIDE', 
      'READM_30_HOSP_WIDE_DEN',
      'H_NUMB_COMP', 
      'H_RESP_RATE_P',
      'HCP_COVID_19',
      'HCP_COVID_19_DEN',
      'H_COMP_1_STAR_RATING', 
      'H_COMP_2_STAR_RATING', 
      'H_COMP_3_STAR_RATING', 
      'H_COMP_5_STAR_RATING', 
      'H_COMP_6_STAR_RATING', 
      'H_COMP_7_STAR_RATING',
      'H_GLOB_STAR_RATING', 
      'H_INDI_STAR_RATING', 
     ]
for l in ls:
    try:
        sas_input_df.drop(labels=[l], axis=1, inplace=True)
    except:
        print(l, 'not present')
        pass
    
ls = list(sas_input_df)
for l in ls:
    if '_DEN' in l:
        print('dropping', l)
        sas_input_df.drop(labels=[l], axis=1, inplace=True)
        
sas_cols_2026 = list(sas_input_df)

ls = ['Hybrid_HWR',
      #'PC_02', #'PC_05', 
      #'ePC_07a', #'ePC_07b',
      #'HH-01', 'HH-02',
      'Hybrid_HWM',
      'O_COMP_1_LINEAR_SCORE',
      'O_COMP_2_LINEAR_SCORE',
      'O_COMP_3_LINEAR_SCORE',
      'O_PATIENT_RATE_LINEAR_SCORE',
      'O_PATIENT_REC_LINEAR_SCORE',
      'H_COMP_1_LINEAR_SCORE',
      'H_COMP_2_LINEAR_SCORE', 
      'H_COMP_3_LINEAR_SCORE', 
      'H_COMP_5_LINEAR_SCORE', 
      'H_COMP_6_LINEAR_SCORE', 
      'H_COMP_7_LINEAR_SCORE', 
      'H_CLEAN_LINEAR_SCORE',  
      'H_QUIET_LINEAR_SCORE', 
      'H_RECMND_LINEAR_SCORE', 
      'H_HSP_RATING_LINEAR_SCORE',
     ]

for l in ls:
    sas_cols_2026.append(l)

sas_cols = list(sas_cols_2026)

sas_input_df = curate(sas_input_df)

print()
print(len(sas_cols_2026), ' features')
print(len(sas_cols_2026) - 1, ' measures:', sorted(sas_cols_2026), '\n')

sas_input_df.head()

dropping IMM_3_DEN
dropping EDAC_30_AMI_DEN
dropping EDAC_30_HF_DEN
dropping OP_18B_DEN
dropping OP_23_DEN
dropping MORT_30_COPD_DEN
dropping HAI_3_DEN_VOL
dropping HAI_5_DEN_PRED
dropping OP_13_DEN
dropping HAI_5_DEN_VOL
dropping HAI_1_DEN_VOL
dropping READM_30_HIP_KNEE_DEN
dropping HAI_6_DEN_VOL
dropping MORT_30_HF_DEN
dropping SAFE_USE_OF_OPIOIDS_DEN
dropping READM_30_COPD_DEN
dropping OP_10_DEN
dropping COMP_HIP_KNEE_DEN
dropping HAI_2_DEN_PRED
dropping OP_29_DEN
dropping OP_22_DEN
dropping MORT_30_AMI_DEN
dropping MORT_30_PN_DEN
dropping OP_36_DEN
dropping MORT_30_CABG_DEN
dropping HAI_1_DEN_PRED
dropping HAI_6_DEN_PRED
dropping READM_30_CABG_DEN
dropping PSI_90_SAFETY_DEN
dropping HAI_3_DEN_PRED
dropping OP_8_DEN
dropping MORT_30_STK_DEN
dropping PSI_4_SURG_COMP_DEN
dropping HAI_2_DEN_VOL
dropping OP_32_DEN
dropping OP_35_ED_DEN
dropping OP_35_ADM_DEN
dropping SEP_1_DEN
dropping EDAC_30_PN_DEN
dropping HAI_4_DEN_VOL
dropping HAI_4_DEN_PRED

53  features
52  measures: ['COMP_HIP_K

,PROVIDER_ID,COMP_HIP_KNEE,EDAC_30_AMI,HAI_3,HAI_6,OP_29,SAFE_USE_OF_OPIOIDS,OP_35_ED,READM_30_CABG,OP_22,OP_36,MORT_30_STK,READM_30_COPD,MORT_30_PN,OP_35_ADM,OP_32,READM_30_HIP_KNEE,MORT_30_CABG,SEP_1,OP_23,OP_18B,MORT_30_AMI,EDAC_30_PN,MORT_30_COPD,OP_13,PSI_90_SAFETY,MORT_30_HF,IMM_3,OP_8,OP_10,EDAC_30_HF,HAI_2,HAI_1,PSI_4_SURG_COMP,HAI_5,HAI_4
0,010001,0.030,-13.8,1.209,0.491,0.47,0.12,4.9,0.102,0.05,1.1,0.137,0.177,0.189,11.9,12.9,0.044,0.038,0.68,NaN,217.0,0.108,14.3,0.079,0.021,0.98,0.105,0.96,0.333,0.054,10.6,0.169,0.496,194.78,0.445,NaN
1,010005,0.027,NaN,0.000,0.631,0.96,0.16,5.5,NaN,0.03,1.9,0.139,0.161,0.231,7.9,14.2,0.039,NaN,0.76,0.69,144.0,0.145,-8.4,0.092,0.028,1.01,0.127,0.72,0.459,0.134,9.8,1.226,0.964,191.14,2.452,NaN
2,010006,0.040,13.4,0.000,0.026,0.85,0.16,NaN,0.115,0.01,1.4,0.142,0.177,0.198,NaN,12.1,0.044,0.052,0.56,NaN,177.0,0.166,4.5,0.102,0.029,1.16,0.122,0.66,NaN,0.108,-4.9,0.000,0.000,201.13,0.371,0.0
3,010007,0.034,NaN,NaN,1.003,0.23,NaN,NaN,NaN,0.04,1.2,NaN,0.190,0.290,NaN,13.4,0.044,NaN,0.24,NaN,129.0,NaN,21.5,0.135,NaN,0.93,0.141,0.26,NaN,0.073,20.9,NaN,NaN,NaN,NaN,NaN
4,010008,NaN,NaN,NaN,NaN,0.67,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.192,NaN,12.9,NaN,NaN,NaN,NaN,121.0,NaN,NaN,NaN,NaN,NaN,NaN,0.39,NaN,0.014,NaN,NaN,NaN,NaN,NaN,NaN


## HAIs

In [4]:
df = pd.read_csv(stars_dir + 'CareCompare/hospitals_11_2025/Healthcare_Associated_Infections-Hospital.csv')
#print(df['Measure ID'].unique())

measures = ['HAI_1_ELIGCASES', 'HAI_1_DOPC', 'HAI_1_SIR', 'HAI_2_ELIGCASES', 'HAI_2_DOPC', 'HAI_2_SIR', 
            'HAI_3_ELIGCASES', 'HAI_3_DOPC', 'HAI_3_SIR', 'HAI_4_ELIGCASES', 'HAI_4_DOPC', 'HAI_4_SIR', 
            'HAI_5_ELIGCASES', 'HAI_5_DOPC', 'HAI_5_SIR', 'HAI_6_ELIGCASES', 'HAI_6_DOPC', 'HAI_6_SIR']

tdf = df[df['Measure ID'].isin(measures + ['End Date', 'Start Date'])]
tdf = tdf.filter(items = ['Measure ID', 'Start Date', 'End Date'])
tdf.drop_duplicates(inplace=True)
dates_df = pd.concat([dates_df, tdf], axis=0)

df = df[df['Measure ID'].isin(measures)]
df = df.filter(items = ['Facility ID', 'Measure ID', 'Score'], axis=1)

hai_df = pd.DataFrame(columns=['Facility ID']) 
for m in measures:
    tdf1 = df[df['Measure ID'] == m]
    tdf2 = pd.DataFrame(columns=['Facility ID', m]) 
    tdf2['Facility ID'] = tdf1['Facility ID'].tolist()
    tdf2[m] = tdf1['Score'].tolist()
    hai_df = hai_df.merge(tdf2, on='Facility ID', how='outer')
    
hai_df.rename(columns={'HAI_1_ELIGCASES': 'HAI_1_DEN_PRED',
                       'HAI_1_DOPC': 'HAI_1_DEN_VOL',
                       'HAI_1_SIR': 'HAI_1',
                       'HAI_2_ELIGCASES': 'HAI_2_DEN_PRED',
                       'HAI_2_DOPC': 'HAI_2_DEN_VOL',
                       'HAI_2_SIR': 'HAI_2',
                       'HAI_3_ELIGCASES': 'HAI_3_DEN_PRED',
                       'HAI_3_DOPC': 'HAI_3_DEN_VOL',
                       'HAI_3_SIR': 'HAI_3',
                       'HAI_4_ELIGCASES': 'HAI_4_DEN_PRED',
                       'HAI_4_DOPC': 'HAI_4_DEN_VOL',
                       'HAI_4_SIR': 'HAI_4',
                       'HAI_5_ELIGCASES': 'HAI_5_DEN_PRED',
                       'HAI_5_DOPC': 'HAI_5_DEN_VOL',
                       'HAI_5_SIR': 'HAI_5',
                       'HAI_6_ELIGCASES': 'HAI_6_DEN_PRED',
                       'HAI_6_DOPC': 'HAI_6_DEN_VOL',
                       'HAI_6_SIR': 'HAI_6',
                       'Facility ID': 'PROVIDER_ID',
                   }, inplace=True)

for c in list(hai_df):
    try:
        sas_cols_2026.remove(c)
    except:
        pass

print(len(sas_cols_2026), 'remaining features:', sorted(sas_cols_2026), '\n')
hai_df = curate(hai_df)

46 remaining features: ['COMP_HIP_KNEE', 'EDAC_30_AMI', 'EDAC_30_HF', 'EDAC_30_PN', 'H_CLEAN_LINEAR_SCORE', 'H_COMP_1_LINEAR_SCORE', 'H_COMP_2_LINEAR_SCORE', 'H_COMP_3_LINEAR_SCORE', 'H_COMP_5_LINEAR_SCORE', 'H_COMP_6_LINEAR_SCORE', 'H_COMP_7_LINEAR_SCORE', 'H_HSP_RATING_LINEAR_SCORE', 'H_QUIET_LINEAR_SCORE', 'H_RECMND_LINEAR_SCORE', 'Hybrid_HWM', 'Hybrid_HWR', 'IMM_3', 'MORT_30_AMI', 'MORT_30_CABG', 'MORT_30_COPD', 'MORT_30_HF', 'MORT_30_PN', 'MORT_30_STK', 'OP_10', 'OP_13', 'OP_18B', 'OP_22', 'OP_23', 'OP_29', 'OP_32', 'OP_35_ADM', 'OP_35_ED', 'OP_36', 'OP_8', 'O_COMP_1_LINEAR_SCORE', 'O_COMP_2_LINEAR_SCORE', 'O_COMP_3_LINEAR_SCORE', 'O_PATIENT_RATE_LINEAR_SCORE', 'O_PATIENT_REC_LINEAR_SCORE', 'PSI_4_SURG_COMP', 'PSI_90_SAFETY', 'READM_30_CABG', 'READM_30_COPD', 'READM_30_HIP_KNEE', 'SAFE_USE_OF_OPIOIDS', 'SEP_1'] 



## Unplanned Hospital Visits


In [5]:
df = pd.read_csv(stars_dir + 'CareCompare/hospitals_11_2025/Unplanned_Hospital_Visits-Hospital.csv')
measures = ['EDAC_30_AMI', 'EDAC_30_HF', 'EDAC_30_PN', 'OP_32', 'OP_35_ADM', 'OP_35_ED', 'OP_36', 
            'READM_30_CABG', 'READM_30_COPD', 'READM_30_HIP_KNEE', 'Hybrid_HWR']

tdf = df[df['Measure ID'].isin(measures + ['End Date', 'Start Date'])]
tdf = tdf.filter(items = ['Measure ID', 'Start Date', 'End Date'])
tdf.drop_duplicates(inplace=True)
dates_df = pd.concat([dates_df, tdf], axis=0)

df = df[df['Measure ID'].isin(measures)]
df = df.filter(items = ['Facility ID', 'Denominator', 'Measure ID', 'Score'], axis=1)

uhv_df = pd.DataFrame(columns=['Facility ID'])
for m in measures:
    tdf1 = df[df['Measure ID'] == m]
    tdf2 = pd.DataFrame(columns=['Facility ID', m]) 
    tdf2['Facility ID'] = tdf1['Facility ID'].tolist()
    tdf2[m] = tdf1['Score'].tolist()
    tdf2[m + '_DEN'] = tdf1['Denominator'].tolist()
    uhv_df = uhv_df.merge(tdf2, on='Facility ID', how='outer')

uhv_df.rename(columns={'Facility ID': 'PROVIDER_ID'}, inplace=True)

for c in list(uhv_df):
    try:
        sas_cols_2026.remove(c)
    except:
        pass
    
print(len(sas_cols_2026), 'remaining features:', sorted(sas_cols_2026), '\n')
uhv_df = curate(uhv_df)

35 remaining features: ['COMP_HIP_KNEE', 'H_CLEAN_LINEAR_SCORE', 'H_COMP_1_LINEAR_SCORE', 'H_COMP_2_LINEAR_SCORE', 'H_COMP_3_LINEAR_SCORE', 'H_COMP_5_LINEAR_SCORE', 'H_COMP_6_LINEAR_SCORE', 'H_COMP_7_LINEAR_SCORE', 'H_HSP_RATING_LINEAR_SCORE', 'H_QUIET_LINEAR_SCORE', 'H_RECMND_LINEAR_SCORE', 'Hybrid_HWM', 'IMM_3', 'MORT_30_AMI', 'MORT_30_CABG', 'MORT_30_COPD', 'MORT_30_HF', 'MORT_30_PN', 'MORT_30_STK', 'OP_10', 'OP_13', 'OP_18B', 'OP_22', 'OP_23', 'OP_29', 'OP_8', 'O_COMP_1_LINEAR_SCORE', 'O_COMP_2_LINEAR_SCORE', 'O_COMP_3_LINEAR_SCORE', 'O_PATIENT_RATE_LINEAR_SCORE', 'O_PATIENT_REC_LINEAR_SCORE', 'PSI_4_SURG_COMP', 'PSI_90_SAFETY', 'SAFE_USE_OF_OPIOIDS', 'SEP_1'] 



## COMPLICATIONS AND DEATHS

In [6]:
df = pd.read_csv(stars_dir + 'CareCompare/hospitals_11_2025/Complications_and_Deaths-Hospital.csv')

measures = ['MORT_30_AMI', 'MORT_30_CABG', 'MORT_30_COPD', 'MORT_30_HF', 
            'MORT_30_PN', 'MORT_30_STK', 'PSI_04', 'COMP_HIP_KNEE',
            'PSI_90', 
            'Hybrid_HWM',
           ]

tdf = df[df['Measure ID'].isin(measures + ['End Date', 'Start Date'])]
tdf = tdf.filter(items = ['Measure ID', 'Start Date', 'End Date'])
tdf.drop_duplicates(inplace=True)
dates_df = pd.concat([dates_df, tdf], axis=0, ignore_index=True)


df = df[df['Measure ID'].isin(measures)]
df = df.filter(items = ['Facility ID', 'Measure ID', 'Score', 'Denominator'], axis=1)

cad_df = pd.DataFrame(columns=['Facility ID'])
for m in measures:
    tdf1 = df[df['Measure ID'] == m]
    tdf2 = pd.DataFrame(columns=['Facility ID', m]) 
    tdf2['Facility ID'] = tdf1['Facility ID'].tolist()
    tdf2[m] = tdf1['Score'].tolist()
    
    tdf2[m + '_DEN'] = tdf1['Denominator'].tolist()
    cad_df = cad_df.merge(tdf2, on='Facility ID', how='outer')
    
cad_df.rename(columns={'Facility ID': 'PROVIDER_ID',
                       'PSI_04': 'PSI_4_SURG_COMP',
                       'PSI_04_DEN': 'PSI_4_SURG_COMP_DEN',
                       'PSI_90': 'PSI_90_SAFETY',
                       'PSI_90_DEN': 'PSI_90_SAFETY_DEN',
                   }, inplace=True)

for c in list(cad_df):
    try:
        sas_cols_2026.remove(c)
    except:
        pass
    
print(len(sas_cols_2026), 'remaining features:', sorted(sas_cols_2026), '\n')
cad_df = curate(cad_df)

25 remaining features: ['H_CLEAN_LINEAR_SCORE', 'H_COMP_1_LINEAR_SCORE', 'H_COMP_2_LINEAR_SCORE', 'H_COMP_3_LINEAR_SCORE', 'H_COMP_5_LINEAR_SCORE', 'H_COMP_6_LINEAR_SCORE', 'H_COMP_7_LINEAR_SCORE', 'H_HSP_RATING_LINEAR_SCORE', 'H_QUIET_LINEAR_SCORE', 'H_RECMND_LINEAR_SCORE', 'IMM_3', 'OP_10', 'OP_13', 'OP_18B', 'OP_22', 'OP_23', 'OP_29', 'OP_8', 'O_COMP_1_LINEAR_SCORE', 'O_COMP_2_LINEAR_SCORE', 'O_COMP_3_LINEAR_SCORE', 'O_PATIENT_RATE_LINEAR_SCORE', 'O_PATIENT_REC_LINEAR_SCORE', 'SAFE_USE_OF_OPIOIDS', 'SEP_1'] 



## TIMELY AND EFFECTIVE CARE

Everything except PC-01, which for 2024 is located in the Maternal Health files of the hospitals data archive

In [7]:
df = pd.read_csv(stars_dir + 'CareCompare/hospitals_11_2025/Timely_and_Effective_Care-Hospital.csv')

measures = ['IMM_3', 'OP_18b', 'OP_22', 'OP_23', 'OP_29', 'SEP_1',
            'SAFE_USE_OF_OPIOIDS', 
           ]

tdf = df[df['Measure ID'].isin(measures + ['End Date', 'Start Date'])]
tdf = tdf.filter(items = ['Measure ID', 'Start Date', 'End Date'])
tdf.drop_duplicates(inplace=True)
dates_df = pd.concat([dates_df, tdf], axis=0, ignore_index=True)


df = df[df['Measure ID'].isin(measures)]
df = df.filter(items = ['Facility ID', 'Sample', 'Measure ID', 'Score'], axis=1)

tec_df = pd.DataFrame(columns=['Facility ID'])
for m in measures:
    tdf1 = df[df['Measure ID'] == m]
    tdf2 = pd.DataFrame(columns=['Facility ID', m]) 
    tdf2['Facility ID'] = tdf1['Facility ID'].tolist()
    tdf2[m] = tdf1['Score'].tolist()
    
    if m == 'HCP_COVID_19':
        pass
    else:
        tdf2[m + '_DEN'] = tdf1['Sample'].tolist()
    
    tec_df = tec_df.merge(tdf2, on='Facility ID', how='outer')
    
tec_df.rename(columns={'Facility ID': 'PROVIDER_ID', 
                       'OP_18b': 'OP_18B',
                       'OP_18b_DEN': 'OP_18B_DEN',
                      }, inplace=True)

for c in list(tec_df):
    try:
        sas_cols_2026.remove(c)
    except:
        pass

print(len(sas_cols_2026), 'remaining features:', sorted(sas_cols_2026), '\n')
tec_df = curate(tec_df)

df_dupes = tec_df[tec_df.duplicated('PROVIDER_ID', keep=False)]

print(df_dupes.shape)
df_dupes.head()

18 remaining features: ['H_CLEAN_LINEAR_SCORE', 'H_COMP_1_LINEAR_SCORE', 'H_COMP_2_LINEAR_SCORE', 'H_COMP_3_LINEAR_SCORE', 'H_COMP_5_LINEAR_SCORE', 'H_COMP_6_LINEAR_SCORE', 'H_COMP_7_LINEAR_SCORE', 'H_HSP_RATING_LINEAR_SCORE', 'H_QUIET_LINEAR_SCORE', 'H_RECMND_LINEAR_SCORE', 'OP_10', 'OP_13', 'OP_8', 'O_COMP_1_LINEAR_SCORE', 'O_COMP_2_LINEAR_SCORE', 'O_COMP_3_LINEAR_SCORE', 'O_PATIENT_RATE_LINEAR_SCORE', 'O_PATIENT_REC_LINEAR_SCORE'] 

(2, 15)


,PROVIDER_ID,IMM_3,IMM_3_DEN,OP_18B,OP_18B_DEN,OP_22,OP_22_DEN,OP_23,OP_23_DEN,OP_29,OP_29_DEN,SEP_1,SEP_1_DEN,SAFE_USE_OF_OPIOIDS,SAFE_USE_OF_OPIOIDS_DEN
0,511315,NaN,NaN,120,774,1,17841,53,15,98,40,59,46,16,310
4659,511315,93,620,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
def collapse_hospitals(df, id_col='PROVIDER_ID'):
    """
    Collapse multiple rows per hospital into a single row by
    taking the first non-null value in each column.
    """

    def first_non_null(series):
        non_null = series.dropna()
        return non_null.iloc[0] if not non_null.empty else np.nan

    collapsed_df = (
        df
        .groupby(id_col, as_index=False)
        .agg(first_non_null)
    )

    return collapsed_df


# --- Apply to your dataframe ---
tec_df = collapse_hospitals(tec_df)

df_dupes = tec_df[tec_df.duplicated('PROVIDER_ID', keep=False)]

print(df_dupes.shape)
df_dupes.head()

(0, 15)


,PROVIDER_ID,IMM_3,IMM_3_DEN,OP_18B,OP_18B_DEN,OP_22,OP_22_DEN,OP_23,OP_23_DEN,OP_29,OP_29_DEN,SEP_1,SEP_1_DEN,SAFE_USE_OF_OPIOIDS,SAFE_USE_OF_OPIOIDS_DEN


In [9]:
df = pd.read_csv(stars_dir + 'CareCompare/hospitals_11_2025/VA_TE.csv')

measures = ['IMM-3', 'OP-18b', 'OP-22', 'OP-23', 'OP-29', 'SEP-1', #'HCP-COVID-19',
           ]
# Does not have 'SAFE_USE_OF_OPIOIDS', 'HH-01', 'HH-02'

tec3_df = pd.DataFrame(columns=['Facility ID'])
for m in measures:
    tdf1 = df[df['Measure ID'] == m]
    tdf2 = pd.DataFrame(columns=['Facility ID', m]) 
    tdf2['Facility ID'] = tdf1['Facility ID'].tolist()
    tdf2[m] = tdf1['Score'].tolist()
    
    tdf2[m + '_DEN'] = tdf1['Sample'].tolist()
    
    tec3_df = tec3_df.merge(tdf2, on='Facility ID', how='outer')
    
tec3_df.rename(columns={'Facility ID': 'PROVIDER_ID', 
                        'OP-18b': 'OP_18B',
                        'OP-18b_DEN': 'OP_18B_DEN',
                        
                        'IMM-3': 'IMM_3', 
                        'IMM-3_DEN': 'IMM_3_DEN', 
                        
                        'OP-22': 'OP_22', 
                        'OP-22_DEN': 'OP_22_DEN', 
                        
                        'OP-23': 'OP_23', 
                        'OP-23_DEN': 'OP_23_DEN', 
                        
                        'OP-29': 'OP_29', 
                        'OP-29_DEN': 'OP_29_DEN', 
                        
                        'SEP-1': 'SEP_1', 
                        'SEP-1_DEN': 'SEP_1_DEN', 
                        
                        #'HCP-COVID-19': 'HCP_COVID_19',
                        #'HCP-COVID-19_DEN': 'HCP_COVID_19_DEN',
                        
                      }, inplace=True)

tec3_df.replace('Not Available', np.nan, inplace=True)
print(tec3_df.shape)
tec3_df = curate(tec3_df)
print(tec3_df.shape)

print(tec_df.shape)
print(tec3_df.shape)
tec_df = pd.concat([tec_df, tec3_df])
print(tec_df.shape)

tec_df.head()

(132, 13)
(132, 13)
(4659, 15)
(132, 13)
(4791, 15)


,PROVIDER_ID,IMM_3,IMM_3_DEN,OP_18B,OP_18B_DEN,OP_22,OP_22_DEN,OP_23,OP_23_DEN,OP_29,OP_29_DEN,SEP_1,SEP_1_DEN,SAFE_USE_OF_OPIOIDS,SAFE_USE_OF_OPIOIDS_DEN
0,010001,93,4625,222,387,5,57084,91,11,72,29,69,150,14,4583
1,010005,59,2856,137,1120,3,58624,27,11,100,210,75,289,15,1859
2,010006,64,2565,150,336,1,44924,67,15,86,85,66,137,15,4350
3,010007,29,358,117,1107,2,12667,Not Available,Not Available,65,52,13,15,15,212
4,010008,31,110,112,350,1,6062,Not Available,Not Available,67,12,Not Available,Not Available,18,65


## TIMELY AND EFFECTIVE CARE

Note: Maternal Health measures (e.g., PC-02, PC-07a, PC-07b) were not included in the Nov 2025 update due to widespread reporting errors

## HCAHPS

In [10]:
df = pd.read_csv(stars_dir + 'CareCompare/hospitals_11_2025/HCAHPS-Hospital.csv')
df.head()

,Facility ID,Facility Name,Address,City/Town,State,ZIP Code,County/Parish,Telephone Number,HCAHPS Measure ID,HCAHPS Question,HCAHPS Answer Description,Patient Survey Star Rating,Patient Survey Star Rating Footnote,HCAHPS Answer Percent,HCAHPS Answer Percent Footnote,HCAHPS Linear Mean Value,Number of Completed Surveys,Number of Completed Surveys Footnote,Survey Response Rate Percent,Survey Response Rate Percent Footnote,Start Date,End Date
0,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,H_COMP_1_A_P,"Patients who reported that their nurses ""Alway...","Nurses ""always"" communicated well",Not Applicable,NaN,75,NaN,Not Applicable,627,NaN,17,NaN,01/01/2024,12/31/2024
1,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,H_COMP_1_SN_P,"Patients who reported that their nurses ""Somet...","Nurses ""sometimes"" or ""never"" communicated well",Not Applicable,NaN,6,NaN,Not Applicable,627,NaN,17,NaN,01/01/2024,12/31/2024
2,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,H_COMP_1_U_P,"Patients who reported that their nurses ""Usual...","Nurses ""usually"" communicated well",Not Applicable,NaN,19,NaN,Not Applicable,627,NaN,17,NaN,01/01/2024,12/31/2024
3,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,H_COMP_1_LINEAR_SCORE,Nurse communication - linear mean score,Nurse communication - linear mean score,Not Applicable,NaN,Not Applicable,NaN,90,627,NaN,17,NaN,01/01/2024,12/31/2024
4,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,H_COMP_1_STAR_RATING,Nurse communication - star rating,Nurse communication - star rating,3,NaN,Not Applicable,NaN,Not Applicable,627,NaN,17,NaN,01/01/2024,12/31/2024


In [11]:
df = pd.read_csv(stars_dir + 'CareCompare/hospitals_11_2025/HCAHPS-Hospital.csv')
measures = df['HCAHPS Measure ID'].unique().tolist()
for m in measures:
    print(m)

H_COMP_1_A_P
H_COMP_1_SN_P
H_COMP_1_U_P
H_COMP_1_LINEAR_SCORE
H_COMP_1_STAR_RATING
H_NURSE_RESPECT_A_P
H_NURSE_RESPECT_SN_P
H_NURSE_RESPECT_U_P
H_NURSE_LISTEN_A_P
H_NURSE_LISTEN_SN_P
H_NURSE_LISTEN_U_P
H_NURSE_EXPLAIN_A_P
H_NURSE_EXPLAIN_SN_P
H_NURSE_EXPLAIN_U_P
H_COMP_2_A_P
H_COMP_2_SN_P
H_COMP_2_U_P
H_COMP_2_LINEAR_SCORE
H_COMP_2_STAR_RATING
H_DOCTOR_RESPECT_A_P
H_DOCTOR_RESPECT_SN_P
H_DOCTOR_RESPECT_U_P
H_DOCTOR_LISTEN_A_P
H_DOCTOR_LISTEN_SN_P
H_DOCTOR_LISTEN_U_P
H_DOCTOR_EXPLAIN_A_P
H_DOCTOR_EXPLAIN_SN_P
H_DOCTOR_EXPLAIN_U_P
H_COMP_3_A_P
H_COMP_3_SN_P
H_COMP_3_U_P
H_COMP_3_LINEAR_SCORE
H_COMP_3_STAR_RATING
H_CALL_BUTTON_A_P
H_CALL_BUTTON_SN_P
H_CALL_BUTTON_U_P
H_BATH_HELP_A_P
H_BATH_HELP_SN_P
H_BATH_HELP_U_P
H_COMP_5_A_P
H_COMP_5_SN_P
H_COMP_5_U_P
H_COMP_5_LINEAR_SCORE
H_COMP_5_STAR_RATING
H_MED_FOR_A_P
H_MED_FOR_SN_P
H_MED_FOR_U_P
H_SIDE_EFFECTS_A_P
H_SIDE_EFFECTS_SN_P
H_SIDE_EFFECTS_U_P
H_COMP_6_N_P
H_COMP_6_Y_P
H_COMP_6_LINEAR_SCORE
H_COMP_6_STAR_RATING
H_DISCH_HELP_N_P
H_DISCH_

In [12]:
tdf = df[df['HCAHPS Measure ID'].isin(measures + ['End Date', 'Start Date'])]
tdf.rename(columns={'HCAHPS Measure ID': 'Measure ID'}, inplace=True)
tdf = tdf.filter(items = ['Measure ID', 'Start Date', 'End Date'])
tdf.drop_duplicates(inplace=True)
dates_df = pd.concat([dates_df, tdf], axis=0, ignore_index=True)

df = df[df['HCAHPS Measure ID'].isin(measures)]
df = df.filter(items = ['Facility ID', 'HCAHPS Measure ID', 'HCAHPS Linear Mean Value', 
                        'Number of Completed Surveys', 'Survey Response Rate Percent'], axis=1)

HCAHPS_df = pd.DataFrame(columns=['Facility ID'])
for i, m in enumerate(measures):
    tdf1 = df[df['HCAHPS Measure ID'] == m]
    tdf2 = pd.DataFrame(columns=['Facility ID', m]) 
    tdf2['Facility ID'] = tdf1['Facility ID'].tolist()
    tdf2[m] = tdf1['HCAHPS Linear Mean Value'].tolist()
    #if i == 0:
    #    tdf2['H_NUMB_COMP'] = tdf1['Number of Completed Surveys'].tolist()
    #    tdf2['H_RESP_RATE_P'] = tdf1['Survey Response Rate Percent'].tolist()
        
    HCAHPS_df = HCAHPS_df.merge(tdf2, on='Facility ID', how='outer')
    

HCAHPS_df.rename(columns={'Facility ID': 'PROVIDER_ID'}, inplace=True)

for c in list(HCAHPS_df):
    try:
        sas_cols_2026.remove(c)
    except:
        pass

print(len(sas_cols_2026), 'remaining features:', sorted(sas_cols_2026), '\n')
HCAHPS_df = curate(HCAHPS_df)

8 remaining features: ['OP_10', 'OP_13', 'OP_8', 'O_COMP_1_LINEAR_SCORE', 'O_COMP_2_LINEAR_SCORE', 'O_COMP_3_LINEAR_SCORE', 'O_PATIENT_RATE_LINEAR_SCORE', 'O_PATIENT_REC_LINEAR_SCORE'] 



#
# OCAHPS

In [13]:
df = pd.read_csv(stars_dir + 'CareCompare/hospitals_11_2025/OQR_OAS_CAHPS_BY_HOSPITAL.csv')
measures = df['OAS CAHPS Measure ID'].unique().tolist()

for m in measures:
    print(m)
    
df.head()

O_COMP_1_LINEAR_SCORE
O_COMP_1_N_P
O_COMP_1_YD_P
O_COMP_1_YS_P
O_COMP_2_LINEAR_SCORE
O_COMP_2_N_P
O_COMP_2_YD_P
O_COMP_2_YS_P
O_COMP_3_LINEAR_SCORE
O_COMP_3_N_P
O_COMP_3_YD_P
O_COMP_3_YS_P
O_PATIENT_RATE_0_6_P
O_PATIENT_RATE_7_8_P
O_PATIENT_RATE_9_10_P
O_PATIENT_RATE_LINEAR_SCORE
O_PATIENT_REC_LINEAR_SCORE
O_PATIENT_REC_NPD_P
O_PATIENT_REC_YD_P
O_PATIENT_REC_YP_P


,Facility ID,Facility Name,Address,City/Town,State,ZIP Code,County/Parish,Telephone Number,OAS CAHPS Measure ID,OAS CAHPS Question,OAS CAHPS Answer Description,OAS CAHPS Answer Percent,OAS CAHPS Answer Percent Footnote,OAS CAHPS Linear Mean Value,Number of Completed Surveys,Number of Completed Surveys Footnote,Survey Response Rate Percent,Survey Response Rate Percent Footnote,Start Date,End Date
0,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,O_COMP_1_LINEAR_SCORE,Facilities and Staff - linear mean score,Facilities and Staff - linear mean score,Not Applicable,NaN,98,185,NaN,17,NaN,01/01/2024,12/31/2024
1,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,O_COMP_1_N_P,"Patients who reported that the facility was ""n...","Facility was ""not"" clean and staff were ""not"" ...",0,NaN,Not Applicable,185,NaN,17,NaN,01/01/2024,12/31/2024
2,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,O_COMP_1_YD_P,"Patients who reported that ""definitely"" the fa...","Facility was ""definitely"" clean and staff were...",97,NaN,Not Applicable,185,NaN,17,NaN,01/01/2024,12/31/2024
3,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,O_COMP_1_YS_P,"Patients who reported ""somewhat"" that the faci...","Facility was ""somewhat"" clean and staff were ""...",3,NaN,Not Applicable,185,NaN,17,NaN,01/01/2024,12/31/2024
4,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,O_COMP_2_LINEAR_SCORE,Communications about your procedure - linear m...,Communications about your procedure - linear m...,Not Applicable,NaN,93,185,NaN,17,NaN,01/01/2024,12/31/2024


In [14]:
measures = [
    'O_COMP_1_LINEAR_SCORE',
    'O_COMP_2_LINEAR_SCORE',
    'O_COMP_3_LINEAR_SCORE',
    'O_PATIENT_RATE_LINEAR_SCORE',
    'O_PATIENT_REC_LINEAR_SCORE',
]

tdf = df[df['OAS CAHPS Measure ID'].isin(measures + ['End Date', 'Start Date'])]
tdf.rename(columns={'OAS CAHPS Measure ID': 'Measure ID'}, inplace=True)
tdf = tdf.filter(items = ['Measure ID', 'Start Date', 'End Date'])
tdf.drop_duplicates(inplace=True)
dates_df = pd.concat([dates_df, tdf], axis=0, ignore_index=True)


df = df[df['OAS CAHPS Measure ID'].isin(measures)]
df = df.filter(items = ['Facility ID', 'OAS CAHPS Measure ID', 'OAS CAHPS Linear Mean Value',
                        'Number of Completed Surveys', 'Survey Response Rate Percent'], axis=1)

OAS_CAHPS_df = pd.DataFrame(columns=['Facility ID'])
for i, m in enumerate(measures):
    tdf1 = df[df['OAS CAHPS Measure ID'] == m]
    tdf2 = pd.DataFrame(columns=['Facility ID', m]) 
    tdf2['Facility ID'] = tdf1['Facility ID'].tolist()
    tdf2[m] = tdf1['OAS CAHPS Linear Mean Value'].tolist()
        
    OAS_CAHPS_df = OAS_CAHPS_df.merge(tdf2, on='Facility ID', how='outer')

OAS_CAHPS_df.rename(columns={'Facility ID': 'PROVIDER_ID'}, inplace=True)

for c in list(OAS_CAHPS_df):
    try:
        sas_cols_2026.remove(c)
    except:
        pass

print(len(sas_cols_2026), 'remaining features:', sorted(sas_cols_2026), '\n')
OAS_CAHPS_df = curate(OAS_CAHPS_df)

3 remaining features: ['OP_10', 'OP_13', 'OP_8'] 



In [15]:
measures = [
    'O_COMP_1_LINEAR_SCORE',
    'O_COMP_2_LINEAR_SCORE',
    'O_COMP_3_LINEAR_SCORE',
    'O_PATIENT_RATE_LINEAR_SCORE',
    'O_PATIENT_REC_LINEAR_SCORE',
]

for l in measures:
    ls = pd.to_numeric(OAS_CAHPS_df[l], errors='coerce')
    print(np.nanmin(ls), np.nanmedian(ls), np.nanmean(ls), np.nanmax(ls))

80.0 98.0 98.13780918727915 100.0
82.0 95.0 95.1387728878895 99.0
88.0 98.0 97.42274333440412 100.0
70.0 94.0 93.92579505300354 99.0
68.0 93.0 92.57436556376486 100.0


## Outpatient Imaging Efficiency


In [16]:
df = pd.read_csv(stars_dir + 'CareCompare/hospitals_11_2025/Outpatient_Imaging_Efficiency-Hospital.csv')
measures = ['OP-13', 'OP-8', 'OP-10']

tdf = df[df['Measure ID'].isin(measures + ['End Date', 'Start Date'])]
tdf = tdf.filter(items = ['Measure ID', 'Start Date', 'End Date'])
tdf.drop_duplicates(inplace=True)
dates_df = pd.concat([dates_df, tdf], axis=0, ignore_index=True)


df = df[df['Measure ID'].isin(measures)]
df = df.filter(items = ['Facility ID', 'Measure ID', 'Score'], axis=1)

oie_df = pd.DataFrame(columns=['Facility ID'])
for m in measures:
    tdf1 = df[df['Measure ID'] == m]
    
    tdf2 = pd.DataFrame(columns=['Facility ID', m]) 
    tdf2['Facility ID'] = tdf1['Facility ID'].tolist()
    tdf2[m] = tdf1['Score'].tolist()
    
    oie_df = oie_df.merge(tdf2, on='Facility ID', how='outer')
    
    
oie_df.rename(columns={'Facility ID': 'PROVIDER_ID',
                       'OP-13': 'OP_13',
                       'OP-8': 'OP_8',
                       'OP-10': 'OP_10',
                   }, inplace=True)

for c in list(oie_df):
    try:
        sas_cols_2026.remove(c)
    except:
        pass

print(len(sas_cols_2026), 'remaining features:', sorted(sas_cols_2026), '\n')
oie_df1 = curate(oie_df)


0 remaining features: [] 



## MERGE DATAFRAME AND COMPARE TO SAS FILE

In [17]:
print(dates_df.shape)
dates_df.to_csv(stars_dir + "2024/measure_dates/11_2025_measure_dates.csv", index=False)
dates_df.head()

(147, 3)


,Measure ID,Start Date,End Date
0,HAI_1_DOPC,01/01/2024,12/31/2024
1,HAI_1_ELIGCASES,01/01/2024,12/31/2024
2,HAI_1_SIR,01/01/2024,12/31/2024
3,HAI_2_DOPC,01/01/2024,12/31/2024
4,HAI_2_ELIGCASES,01/01/2024,12/31/2024


In [18]:
main_df = tec_df.merge(cad_df, on='PROVIDER_ID', how='outer')
main_df = main_df.merge(HCAHPS_df, on='PROVIDER_ID', how='outer')
main_df = main_df.merge(OAS_CAHPS_df, on='PROVIDER_ID', how='outer')
main_df = main_df.merge(uhv_df, on='PROVIDER_ID', how='outer')
main_df = main_df.merge(hai_df, on='PROVIDER_ID', how='outer')
main_df = main_df.merge(oie_df1, on='PROVIDER_ID', how='outer')

for c in list(main_df):
    try:
        sas_cols_2026.remove(c)
    except:
        pass

print(len(sas_cols_2026), 'remaining features:', sorted(sas_cols_2026), '\n')
print(main_df.shape)
main_df.head()

0 remaining features: [] 

(4791, 176)


,PROVIDER_ID,IMM_3,IMM_3_DEN,OP_18B,OP_18B_DEN,OP_22,OP_22_DEN,OP_23,OP_23_DEN,OP_29,OP_29_DEN,SEP_1,SEP_1_DEN,SAFE_USE_OF_OPIOIDS,SAFE_USE_OF_OPIOIDS_DEN,MORT_30_AMI,MORT_30_AMI_DEN,MORT_30_CABG,MORT_30_CABG_DEN,MORT_30_COPD,MORT_30_COPD_DEN,MORT_30_HF,MORT_30_HF_DEN,MORT_30_PN,MORT_30_PN_DEN,MORT_30_STK,MORT_30_STK_DEN,PSI_4_SURG_COMP,PSI_4_SURG_COMP_DEN,COMP_HIP_KNEE,COMP_HIP_KNEE_DEN,PSI_90_SAFETY,PSI_90_SAFETY_DEN,Hybrid_HWM,Hybrid_HWM_DEN,H_COMP_1_A_P,H_COMP_1_SN_P,H_COMP_1_U_P,H_COMP_1_LINEAR_SCORE,H_COMP_1_STAR_RATING,H_NURSE_RESPECT_A_P,H_NURSE_RESPECT_SN_P,H_NURSE_RESPECT_U_P,H_NURSE_LISTEN_A_P,H_NURSE_LISTEN_SN_P,H_NURSE_LISTEN_U_P,H_NURSE_EXPLAIN_A_P,H_NURSE_EXPLAIN_SN_P,H_NURSE_EXPLAIN_U_P,H_COMP_2_A_P,H_COMP_2_SN_P,H_COMP_2_U_P,H_COMP_2_LINEAR_SCORE,H_COMP_2_STAR_RATING,H_DOCTOR_RESPECT_A_P,H_DOCTOR_RESPECT_SN_P,H_DOCTOR_RESPECT_U_P,H_DOCTOR_LISTEN_A_P,H_DOCTOR_LISTEN_SN_P,H_DOCTOR_LISTEN_U_P,H_DOCTOR_EXPLAIN_A_P,H_DOCTOR_EXPLAIN_SN_P,H_DOCTOR_EXPLAIN_U_P,H_COMP_3_A_P,H_COMP_3_SN_P,H_COMP_3_U_P,H_COMP_3_LINEAR_SCORE,H_COMP_3_STAR_RATING,H_CALL_BUTTON_A_P,H_CALL_BUTTON_SN_P,H_CALL_BUTTON_U_P,H_BATH_HELP_A_P,H_BATH_HELP_SN_P,H_BATH_HELP_U_P,H_COMP_5_A_P,H_COMP_5_SN_P,H_COMP_5_U_P,H_COMP_5_LINEAR_SCORE,H_COMP_5_STAR_RATING,H_MED_FOR_A_P,H_MED_FOR_SN_P,H_MED_FOR_U_P,H_SIDE_EFFECTS_A_P,H_SIDE_EFFECTS_SN_P,H_SIDE_EFFECTS_U_P,H_COMP_6_N_P,H_COMP_6_Y_P,H_COMP_6_LINEAR_SCORE,H_COMP_6_STAR_RATING,H_DISCH_HELP_N_P,H_DISCH_HELP_Y_P,H_SYMPTOMS_N_P,H_SYMPTOMS_Y_P,H_COMP_7_A,H_COMP_7_D_SD,H_COMP_7_SA,H_COMP_7_LINEAR_SCORE,H_COMP_7_STAR_RATING,H_CT_PREFER_A,H_CT_PREFER_D_SD,H_CT_PREFER_SA,H_CT_UNDER_A,H_CT_UNDER_D_SD,H_CT_UNDER_SA,H_CT_MED_A,H_CT_MED_D_SD,H_CT_MED_SA,H_CLEAN_HSP_A_P,H_CLEAN_HSP_SN_P,H_CLEAN_HSP_U_P,H_CLEAN_LINEAR_SCORE,H_CLEAN_STAR_RATING,H_QUIET_HSP_A_P,H_QUIET_HSP_SN_P,H_QUIET_HSP_U_P,H_QUIET_LINEAR_SCORE,H_QUIET_STAR_RATING,H_HSP_RATING_0_6,H_HSP_RATING_7_8,H_HSP_RATING_9_10,H_HSP_RATING_LINEAR_SCORE,H_HSP_RATING_STAR_RATING,H_RECMND_DN,H_RECMND_DY,H_RECMND_PY,H_RECMND_LINEAR_SCORE,H_RECMND_STAR_RATING,H_STAR_RATING,O_COMP_1_LINEAR_SCORE,O_COMP_2_LINEAR_SCORE,O_COMP_3_LINEAR_SCORE,O_PATIENT_RATE_LINEAR_SCORE,O_PATIENT_REC_LINEAR_SCORE,EDAC_30_AMI,EDAC_30_AMI_DEN,EDAC_30_HF,EDAC_30_HF_DEN,EDAC_30_PN,EDAC_30_PN_DEN,OP_32,OP_32_DEN,OP_35_ADM,OP_35_ADM_DEN,OP_35_ED,OP_35_ED_DEN,OP_36,OP_36_DEN,READM_30_CABG,READM_30_CABG_DEN,READM_30_COPD,READM_30_COPD_DEN,READM_30_HIP_KNEE,READM_30_HIP_KNEE_DEN,Hybrid_HWR,Hybrid_HWR_DEN,HAI_1_DEN_PRED,HAI_1_DEN_VOL,HAI_1,HAI_2_DEN_PRED,HAI_2_DEN_VOL,HAI_2,HAI_3_DEN_PRED,HAI_3_DEN_VOL,HAI_3,HAI_4_DEN_PRED,HAI_4_DEN_VOL,HAI_4,HAI_5_DEN_PRED,HAI_5_DEN_VOL,HAI_5,HAI_6_DEN_PRED,HAI_6_DEN_VOL,HAI_6,OP_13,OP_8,OP_10
0,010001,93,4625,222,387,5,57084,91,11,72,29,69,150,14,4583,11.4,270,3,144,9.4,112,10.2,583,18.4,517,13.5,395,203.00,118,3.2,27,0.95,Not Applicable,4.5,1835,Not Applicable,Not Applicable,Not Applicable,90,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,92,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,82,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,79,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,87,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,82,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,85,Not Applicable,Not Applicable,Not Applicable,Not Applicable,88,Not Applicable,Not

In [19]:
df_dupes = main_df[main_df.duplicated('PROVIDER_ID', keep=False)]

print(df_dupes.shape)
df_dupes.head()

(0, 176)


,PROVIDER_ID,IMM_3,IMM_3_DEN,OP_18B,OP_18B_DEN,OP_22,OP_22_DEN,OP_23,OP_23_DEN,OP_29,OP_29_DEN,SEP_1,SEP_1_DEN,SAFE_USE_OF_OPIOIDS,SAFE_USE_OF_OPIOIDS_DEN,MORT_30_AMI,MORT_30_AMI_DEN,MORT_30_CABG,MORT_30_CABG_DEN,MORT_30_COPD,MORT_30_COPD_DEN,MORT_30_HF,MORT_30_HF_DEN,MORT_30_PN,MORT_30_PN_DEN,MORT_30_STK,MORT_30_STK_DEN,PSI_4_SURG_COMP,PSI_4_SURG_COMP_DEN,COMP_HIP_KNEE,COMP_HIP_KNEE_DEN,PSI_90_SAFETY,PSI_90_SAFETY_DEN,Hybrid_HWM,Hybrid_HWM_DEN,H_COMP_1_A_P,H_COMP_1_SN_P,H_COMP_1_U_P,H_COMP_1_LINEAR_SCORE,H_COMP_1_STAR_RATING,H_NURSE_RESPECT_A_P,H_NURSE_RESPECT_SN_P,H_NURSE_RESPECT_U_P,H_NURSE_LISTEN_A_P,H_NURSE_LISTEN_SN_P,H_NURSE_LISTEN_U_P,H_NURSE_EXPLAIN_A_P,H_NURSE_EXPLAIN_SN_P,H_NURSE_EXPLAIN_U_P,H_COMP_2_A_P,H_COMP_2_SN_P,H_COMP_2_U_P,H_COMP_2_LINEAR_SCORE,H_COMP_2_STAR_RATING,H_DOCTOR_RESPECT_A_P,H_DOCTOR_RESPECT_SN_P,H_DOCTOR_RESPECT_U_P,H_DOCTOR_LISTEN_A_P,H_DOCTOR_LISTEN_SN_P,H_DOCTOR_LISTEN_U_P,H_DOCTOR_EXPLAIN_A_P,H_DOCTOR_EXPLAIN_SN_P,H_DOCTOR_EXPLAIN_U_P,H_COMP_3_A_P,H_COMP_3_SN_P,H_COMP_3_U_P,H_COMP_3_LINEAR_SCORE,H_COMP_3_STAR_RATING,H_CALL_BUTTON_A_P,H_CALL_BUTTON_SN_P,H_CALL_BUTTON_U_P,H_BATH_HELP_A_P,H_BATH_HELP_SN_P,H_BATH_HELP_U_P,H_COMP_5_A_P,H_COMP_5_SN_P,H_COMP_5_U_P,H_COMP_5_LINEAR_SCORE,H_COMP_5_STAR_RATING,H_MED_FOR_A_P,H_MED_FOR_SN_P,H_MED_FOR_U_P,H_SIDE_EFFECTS_A_P,H_SIDE_EFFECTS_SN_P,H_SIDE_EFFECTS_U_P,H_COMP_6_N_P,H_COMP_6_Y_P,H_COMP_6_LINEAR_SCORE,H_COMP_6_STAR_RATING,H_DISCH_HELP_N_P,H_DISCH_HELP_Y_P,H_SYMPTOMS_N_P,H_SYMPTOMS_Y_P,H_COMP_7_A,H_COMP_7_D_SD,H_COMP_7_SA,H_COMP_7_LINEAR_SCORE,H_COMP_7_STAR_RATING,H_CT_PREFER_A,H_CT_PREFER_D_SD,H_CT_PREFER_SA,H_CT_UNDER_A,H_CT_UNDER_D_SD,H_CT_UNDER_SA,H_CT_MED_A,H_CT_MED_D_SD,H_CT_MED_SA,H_CLEAN_HSP_A_P,H_CLEAN_HSP_SN_P,H_CLEAN_HSP_U_P,H_CLEAN_LINEAR_SCORE,H_CLEAN_STAR_RATING,H_QUIET_HSP_A_P,H_QUIET_HSP_SN_P,H_QUIET_HSP_U_P,H_QUIET_LINEAR_SCORE,H_QUIET_STAR_RATING,H_HSP_RATING_0_6,H_HSP_RATING_7_8,H_HSP_RATING_9_10,H_HSP_RATING_LINEAR_SCORE,H_HSP_RATING_STAR_RATING,H_RECMND_DN,H_RECMND_DY,H_RECMND_PY,H_RECMND_LINEAR_SCORE,H_RECMND_STAR_RATING,H_STAR_RATING,O_COMP_1_LINEAR_SCORE,O_COMP_2_LINEAR_SCORE,O_COMP_3_LINEAR_SCORE,O_PATIENT_RATE_LINEAR_SCORE,O_PATIENT_REC_LINEAR_SCORE,EDAC_30_AMI,EDAC_30_AMI_DEN,EDAC_30_HF,EDAC_30_HF_DEN,EDAC_30_PN,EDAC_30_PN_DEN,OP_32,OP_32_DEN,OP_35_ADM,OP_35_ADM_DEN,OP_35_ED,OP_35_ED_DEN,OP_36,OP_36_DEN,READM_30_CABG,READM_30_CABG_DEN,READM_30_COPD,READM_30_COPD_DEN,READM_30_HIP_KNEE,READM_30_HIP_KNEE_DEN,Hybrid_HWR,Hybrid_HWR_DEN,HAI_1_DEN_PRED,HAI_1_DEN_VOL,HAI_1,HAI_2_DEN_PRED,HAI_2_DEN_VOL,HAI_2,HAI_3_DEN_PRED,HAI_3_DEN_VOL,HAI_3,HAI_4_DEN_PRED,HAI_4_DEN_VOL,HAI_4,HAI_5_DEN_PRED,HAI_5_DEN_VOL,HAI_5,HAI_6_DEN_PRED,HAI_6_DEN_VOL,HAI_6,OP_13,OP_8,OP_10


In [20]:

cols = []
for col in list(main_df):
    if 'PROVIDER_ID' in col:
        continue
    elif '_DEN' in col:
        continue
    else:
        cols.append(col)
        
print(sorted(cols), '\n')
print(len(cols), 'measures')

['COMP_HIP_KNEE', 'EDAC_30_AMI', 'EDAC_30_HF', 'EDAC_30_PN', 'HAI_1', 'HAI_2', 'HAI_3', 'HAI_4', 'HAI_5', 'HAI_6', 'H_BATH_HELP_A_P', 'H_BATH_HELP_SN_P', 'H_BATH_HELP_U_P', 'H_CALL_BUTTON_A_P', 'H_CALL_BUTTON_SN_P', 'H_CALL_BUTTON_U_P', 'H_CLEAN_HSP_A_P', 'H_CLEAN_HSP_SN_P', 'H_CLEAN_HSP_U_P', 'H_CLEAN_LINEAR_SCORE', 'H_CLEAN_STAR_RATING', 'H_COMP_1_A_P', 'H_COMP_1_LINEAR_SCORE', 'H_COMP_1_SN_P', 'H_COMP_1_STAR_RATING', 'H_COMP_1_U_P', 'H_COMP_2_A_P', 'H_COMP_2_LINEAR_SCORE', 'H_COMP_2_SN_P', 'H_COMP_2_STAR_RATING', 'H_COMP_2_U_P', 'H_COMP_3_A_P', 'H_COMP_3_LINEAR_SCORE', 'H_COMP_3_SN_P', 'H_COMP_3_STAR_RATING', 'H_COMP_3_U_P', 'H_COMP_5_A_P', 'H_COMP_5_LINEAR_SCORE', 'H_COMP_5_SN_P', 'H_COMP_5_STAR_RATING', 'H_COMP_5_U_P', 'H_COMP_6_LINEAR_SCORE', 'H_COMP_6_N_P', 'H_COMP_6_STAR_RATING', 'H_COMP_6_Y_P', 'H_COMP_7_A', 'H_COMP_7_D_SD', 'H_COMP_7_LINEAR_SCORE', 'H_COMP_7_SA', 'H_COMP_7_STAR_RATING', 'H_CT_MED_A', 'H_CT_MED_D_SD', 'H_CT_MED_SA', 'H_CT_PREFER_A', 'H_CT_PREFER_D_SD', 'H_CT_P

In [21]:
print(main_df.shape)
main_df.dropna(how='all', subset = cols, inplace=True)
print(main_df.shape)

labs = ['READM_30_HIP_KNEE', 'READM_30_COPD', 'MORT_30_STK', 'MORT_30_PN',
        'MORT_30_HF', 'MORT_30_COPD', 'MORT_30_AMI', 'COMP_HIP_KNEE', 'OP_22',
        'OP_23', 'OP_29', 'IMM_3', 'SEP_1', 'MORT_30_CABG',
        'READM_30_CABG', 'OP_8',
        'OP_10', 'OP_13', 
        'SAFE_USE_OF_OPIOIDS',
        'Hybrid_HWR',
        'Hybrid_HWM',
       ]

summary = {}
for col in labs:
    series = main_df[col]
    
    # Count NaNs and "Not Available"
    n_nans = series.isna().sum()
    n_notavail = (series == 'Not Available').sum()
    
    # Coerce to numeric (turns "Not Available" into NaN)
    numeric = pd.to_numeric(series, errors='coerce')
    
    # Count valid numeric entries
    n_numeric = numeric.notna().sum()
    
    summary[col] = {
        'min': numeric.min(skipna=True),
        'max': numeric.max(skipna=True),
        'mean': numeric.mean(skipna=True),
        'median': numeric.median(skipna=True),
        'numeric_count': n_numeric,
        'NaN_count': n_nans,
        "'Not Available'_count": n_notavail
    }

summary_df = pd.DataFrame(summary).T
print(summary_df)


(4791, 176)
(4791, 176)
                      min    max       mean  median  numeric_count  NaN_count  \
READM_30_HIP_KNEE     2.4    7.8   4.856458     4.8         1626.0        0.0   
READM_30_COPD        15.7   23.9  18.228545    18.1         2701.0        0.0   
MORT_30_STK           8.1   22.1  13.235634    13.1         2217.0        0.0   
MORT_30_PN            7.6   32.2  16.353067    16.2         3652.0        0.0   
MORT_30_HF            5.0   20.2  11.575185    11.5         3107.0        0.0   
MORT_30_COPD          4.5   16.5   8.877648     8.7         2662.0        0.0   
MORT_30_AMI           6.7   17.1  12.140317    12.1         1957.0        0.0   
COMP_HIP_KNEE         1.4    9.3   3.615751     3.5         1657.0        0.0   
OP_22                 0.0  100.0   1.972187     1.0         3955.0       24.0   
OP_23                 0.0  100.0  70.691432    74.0         1494.0      130.0   
OP_29                 0.0  100.0  91.837479    97.0         2935.0       33.0   
IMM_

In [22]:
print(main_df.shape)
main_df.dropna(how='all', subset = cols, inplace=True)
print(main_df.shape)

ls = ['READM_30_HIP_KNEE', 'READM_30_COPD', 'MORT_30_STK', 'MORT_30_PN',
      'MORT_30_HF', 'MORT_30_COPD', 'MORT_30_AMI', 'COMP_HIP_KNEE', 'OP_22',
      'OP_23', 'OP_29', 'IMM_3', 'SEP_1', 'MORT_30_CABG',
      'READM_30_CABG', 'OP_8',
      'OP_10', 'OP_13', 
      'SAFE_USE_OF_OPIOIDS',
      'Hybrid_HWM',
      'Hybrid_HWR', 
     ]

for col in list(main_df):
    if col != 'PROVIDER_ID':
        main_df[col] = pd.to_numeric(main_df[col], errors='coerce').astype(float)
        
for l in ls: 
    main_df[l] = main_df[l] * 0.01


(4791, 176)
(4791, 176)


# Notes
52 measures in 2026

### 1 measure was renamed:

READM_30_HOSP_WIDE: Rate of readmission after discharge from hospital (hospital-wide)

... renamed to ...

Hybrid_HWR: Hybrid Hospital-Wide All-Cause Readmission Rate

### Lost 2 measures:

PC_01: Percentage of mothers whose deliveries were scheduled too early (1-2 weeks early), when a scheduled
delivery wasn’t medically necessary

HCP_COVID_19: Percentage of employees that were COVID-19 vaccinated 

### Including 6 new measures for 2026:

Hybrid_HWM: Hybrid Hospital-Wide All-Cause Risk Standardized Mortality Rate

OQR_OAS_CAHPS:
O_COMP_1_LINEAR_SCORE
O_COMP_2_LINEAR_SCORE
O_COMP_3_LINEAR_SCORE
O_PATIENT_RATE_LINEAR_SCORE
O_PATIENT_REC_LINEAR_SCORE

### HCAHPS measure updates

H_INDI_STAR_RATING replaced with H_QUIET_LINEAR_SCORE and H_CLEAN_LINEAR_SCORE
H_GLOB_STAR_RATING replaced with H_HSP_RATING_LINEAR_SCORE and H_RECMND_LINEAR_SCORE


In [23]:
prvdrs = []
for p in main_df['PROVIDER_ID'].tolist():
    if 'F' in p:
        p = p[:-1]
        p = p + '666666'
    prvdrs.append(p)

main_df['PROVIDER_ID'] = prvdrs

main_df['PROVIDER_ID'] = pd.to_numeric(main_df['PROVIDER_ID'], errors='coerce')
main_df.sort_values(by=['PROVIDER_ID'], ascending = True, inplace = True)
main_df.to_csv(stars_dir + "Reproduce_Stars_Input/2026/Input_File/data_for_2026_prognostications_from_Nov2025.csv", 
               index=False)


In [24]:
for l in main_df.columns:
    if main_df[l].isna().all():
        print(l)

PSI_90_SAFETY_DEN
H_COMP_1_A_P
H_COMP_1_SN_P
H_COMP_1_U_P
H_COMP_1_STAR_RATING
H_NURSE_RESPECT_A_P
H_NURSE_RESPECT_SN_P
H_NURSE_RESPECT_U_P
H_NURSE_LISTEN_A_P
H_NURSE_LISTEN_SN_P
H_NURSE_LISTEN_U_P
H_NURSE_EXPLAIN_A_P
H_NURSE_EXPLAIN_SN_P
H_NURSE_EXPLAIN_U_P
H_COMP_2_A_P
H_COMP_2_SN_P
H_COMP_2_U_P
H_COMP_2_STAR_RATING
H_DOCTOR_RESPECT_A_P
H_DOCTOR_RESPECT_SN_P
H_DOCTOR_RESPECT_U_P
H_DOCTOR_LISTEN_A_P
H_DOCTOR_LISTEN_SN_P
H_DOCTOR_LISTEN_U_P
H_DOCTOR_EXPLAIN_A_P
H_DOCTOR_EXPLAIN_SN_P
H_DOCTOR_EXPLAIN_U_P
H_COMP_3_A_P
H_COMP_3_SN_P
H_COMP_3_U_P
H_COMP_3_STAR_RATING
H_CALL_BUTTON_A_P
H_CALL_BUTTON_SN_P
H_CALL_BUTTON_U_P
H_BATH_HELP_A_P
H_BATH_HELP_SN_P
H_BATH_HELP_U_P
H_COMP_5_A_P
H_COMP_5_SN_P
H_COMP_5_U_P
H_COMP_5_STAR_RATING
H_MED_FOR_A_P
H_MED_FOR_SN_P
H_MED_FOR_U_P
H_SIDE_EFFECTS_A_P
H_SIDE_EFFECTS_SN_P
H_SIDE_EFFECTS_U_P
H_COMP_6_N_P
H_COMP_6_Y_P
H_COMP_6_STAR_RATING
H_DISCH_HELP_N_P
H_DISCH_HELP_Y_P
H_SYMPTOMS_N_P
H_SYMPTOMS_Y_P
H_COMP_7_A
H_COMP_7_D_SD
H_COMP_7_SA
H_COMP_7_STAR_RA

In [25]:
main_df.head()

,PROVIDER_ID,IMM_3,IMM_3_DEN,OP_18B,OP_18B_DEN,OP_22,OP_22_DEN,OP_23,OP_23_DEN,OP_29,OP_29_DEN,SEP_1,SEP_1_DEN,SAFE_USE_OF_OPIOIDS,SAFE_USE_OF_OPIOIDS_DEN,MORT_30_AMI,MORT_30_AMI_DEN,MORT_30_CABG,MORT_30_CABG_DEN,MORT_30_COPD,MORT_30_COPD_DEN,MORT_30_HF,MORT_30_HF_DEN,MORT_30_PN,MORT_30_PN_DEN,MORT_30_STK,MORT_30_STK_DEN,PSI_4_SURG_COMP,PSI_4_SURG_COMP_DEN,COMP_HIP_KNEE,COMP_HIP_KNEE_DEN,PSI_90_SAFETY,PSI_90_SAFETY_DEN,Hybrid_HWM,Hybrid_HWM_DEN,H_COMP_1_A_P,H_COMP_1_SN_P,H_COMP_1_U_P,H_COMP_1_LINEAR_SCORE,H_COMP_1_STAR_RATING,H_NURSE_RESPECT_A_P,H_NURSE_RESPECT_SN_P,H_NURSE_RESPECT_U_P,H_NURSE_LISTEN_A_P,H_NURSE_LISTEN_SN_P,H_NURSE_LISTEN_U_P,H_NURSE_EXPLAIN_A_P,H_NURSE_EXPLAIN_SN_P,H_NURSE_EXPLAIN_U_P,H_COMP_2_A_P,H_COMP_2_SN_P,H_COMP_2_U_P,H_COMP_2_LINEAR_SCORE,H_COMP_2_STAR_RATING,H_DOCTOR_RESPECT_A_P,H_DOCTOR_RESPECT_SN_P,H_DOCTOR_RESPECT_U_P,H_DOCTOR_LISTEN_A_P,H_DOCTOR_LISTEN_SN_P,H_DOCTOR_LISTEN_U_P,H_DOCTOR_EXPLAIN_A_P,H_DOCTOR_EXPLAIN_SN_P,H_DOCTOR_EXPLAIN_U_P,H_COMP_3_A_P,H_COMP_3_SN_P,H_COMP_3_U_P,H_COMP_3_LINEAR_SCORE,H_COMP_3_STAR_RATING,H_CALL_BUTTON_A_P,H_CALL_BUTTON_SN_P,H_CALL_BUTTON_U_P,H_BATH_HELP_A_P,H_BATH_HELP_SN_P,H_BATH_HELP_U_P,H_COMP_5_A_P,H_COMP_5_SN_P,H_COMP_5_U_P,H_COMP_5_LINEAR_SCORE,H_COMP_5_STAR_RATING,H_MED_FOR_A_P,H_MED_FOR_SN_P,H_MED_FOR_U_P,H_SIDE_EFFECTS_A_P,H_SIDE_EFFECTS_SN_P,H_SIDE_EFFECTS_U_P,H_COMP_6_N_P,H_COMP_6_Y_P,H_COMP_6_LINEAR_SCORE,H_COMP_6_STAR_RATING,H_DISCH_HELP_N_P,H_DISCH_HELP_Y_P,H_SYMPTOMS_N_P,H_SYMPTOMS_Y_P,H_COMP_7_A,H_COMP_7_D_SD,H_COMP_7_SA,H_COMP_7_LINEAR_SCORE,H_COMP_7_STAR_RATING,H_CT_PREFER_A,H_CT_PREFER_D_SD,H_CT_PREFER_SA,H_CT_UNDER_A,H_CT_UNDER_D_SD,H_CT_UNDER_SA,H_CT_MED_A,H_CT_MED_D_SD,H_CT_MED_SA,H_CLEAN_HSP_A_P,H_CLEAN_HSP_SN_P,H_CLEAN_HSP_U_P,H_CLEAN_LINEAR_SCORE,H_CLEAN_STAR_RATING,H_QUIET_HSP_A_P,H_QUIET_HSP_SN_P,H_QUIET_HSP_U_P,H_QUIET_LINEAR_SCORE,H_QUIET_STAR_RATING,H_HSP_RATING_0_6,H_HSP_RATING_7_8,H_HSP_RATING_9_10,H_HSP_RATING_LINEAR_SCORE,H_HSP_RATING_STAR_RATING,H_RECMND_DN,H_RECMND_DY,H_RECMND_PY,H_RECMND_LINEAR_SCORE,H_RECMND_STAR_RATING,H_STAR_RATING,O_COMP_1_LINEAR_SCORE,O_COMP_2_LINEAR_SCORE,O_COMP_3_LINEAR_SCORE,O_PATIENT_RATE_LINEAR_SCORE,O_PATIENT_REC_LINEAR_SCORE,EDAC_30_AMI,EDAC_30_AMI_DEN,EDAC_30_HF,EDAC_30_HF_DEN,EDAC_30_PN,EDAC_30_PN_DEN,OP_32,OP_32_DEN,OP_35_ADM,OP_35_ADM_DEN,OP_35_ED,OP_35_ED_DEN,OP_36,OP_36_DEN,READM_30_CABG,READM_30_CABG_DEN,READM_30_COPD,READM_30_COPD_DEN,READM_30_HIP_KNEE,READM_30_HIP_KNEE_DEN,Hybrid_HWR,Hybrid_HWR_DEN,HAI_1_DEN_PRED,HAI_1_DEN_VOL,HAI_1,HAI_2_DEN_PRED,HAI_2_DEN_VOL,HAI_2,HAI_3_DEN_PRED,HAI_3_DEN_VOL,HAI_3,HAI_4_DEN_PRED,HAI_4_DEN_VOL,HAI_4,HAI_5_DEN_PRED,HAI_5_DEN_VOL,HAI_5,HAI_6_DEN_PRED,HAI_6_DEN_VOL,HAI_6,OP_13,OP_8,OP_10
0,10001,0.93,4625.0,222.0,387.0,0.05,57084.0,0.91,11.0,0.72,29.0,0.69,150.0,0.14,4583.0,0.114,270.0,0.030,144.0,0.094,112.0,0.102,583.0,0.184,517.0,0.135,395.0,203.00,118.0,0.032,27.0,0.95,NaN,0.045,1835.0,NaN,NaN,NaN,90.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,92.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,79.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,87.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,85.0,NaN,NaN,NaN,NaN,88.0,NaN,NaN,NaN,NaN,89.0,NaN,NaN,NaN,NaN,91.0,NaN,NaN,98.0,93.0,97.0,94.0,94.0,-15.6,273.0,-1.1,652.0,17.4,507.0,12.8,218.0,8.5,261.0,5.7,261.0,0.8,647.0,0.101,137.0,0.180,122.0,0.048,25.0,0.151,2824.0,9.440,8935.0,0.530,23.350,16255.0,0.086,6.562,229.0,0.457,0.903,97.0,NaN,10.937,109019.0,0.183,68.076,109019.0,0.382,0.038,0.308,0.053
1,10005,0.59,2856.0,137.0,1120.0,0.03,58624.0,0.27,11.0,1.00,210.0,0.75,289.0,0.15,1859.0,NaN,NaN,NaN,NaN,0.089,126.0,0.141,158.0,0.212,285.0,0.129,89.0,184.79,27.0,0.030,104.0,0.97,NaN,0.046,698.0,NaN,NaN,NaN,91.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,93.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88.0,NaN,NaN,

In [26]:
raw_data = pd.read_csv(stars_dir + "Reproduce_Stars_Input/2026/Input_File/data_for_2026_prognostications_from_Nov2025.csv")
raw_data.head()

df_dupes = raw_data[raw_data.duplicated('PROVIDER_ID', keep=False)]

print(df_dupes.shape)
df_dupes.head()

(0, 176)


,PROVIDER_ID,IMM_3,IMM_3_DEN,OP_18B,OP_18B_DEN,OP_22,OP_22_DEN,OP_23,OP_23_DEN,OP_29,OP_29_DEN,SEP_1,SEP_1_DEN,SAFE_USE_OF_OPIOIDS,SAFE_USE_OF_OPIOIDS_DEN,MORT_30_AMI,MORT_30_AMI_DEN,MORT_30_CABG,MORT_30_CABG_DEN,MORT_30_COPD,MORT_30_COPD_DEN,MORT_30_HF,MORT_30_HF_DEN,MORT_30_PN,MORT_30_PN_DEN,MORT_30_STK,MORT_30_STK_DEN,PSI_4_SURG_COMP,PSI_4_SURG_COMP_DEN,COMP_HIP_KNEE,COMP_HIP_KNEE_DEN,PSI_90_SAFETY,PSI_90_SAFETY_DEN,Hybrid_HWM,Hybrid_HWM_DEN,H_COMP_1_A_P,H_COMP_1_SN_P,H_COMP_1_U_P,H_COMP_1_LINEAR_SCORE,H_COMP_1_STAR_RATING,H_NURSE_RESPECT_A_P,H_NURSE_RESPECT_SN_P,H_NURSE_RESPECT_U_P,H_NURSE_LISTEN_A_P,H_NURSE_LISTEN_SN_P,H_NURSE_LISTEN_U_P,H_NURSE_EXPLAIN_A_P,H_NURSE_EXPLAIN_SN_P,H_NURSE_EXPLAIN_U_P,H_COMP_2_A_P,H_COMP_2_SN_P,H_COMP_2_U_P,H_COMP_2_LINEAR_SCORE,H_COMP_2_STAR_RATING,H_DOCTOR_RESPECT_A_P,H_DOCTOR_RESPECT_SN_P,H_DOCTOR_RESPECT_U_P,H_DOCTOR_LISTEN_A_P,H_DOCTOR_LISTEN_SN_P,H_DOCTOR_LISTEN_U_P,H_DOCTOR_EXPLAIN_A_P,H_DOCTOR_EXPLAIN_SN_P,H_DOCTOR_EXPLAIN_U_P,H_COMP_3_A_P,H_COMP_3_SN_P,H_COMP_3_U_P,H_COMP_3_LINEAR_SCORE,H_COMP_3_STAR_RATING,H_CALL_BUTTON_A_P,H_CALL_BUTTON_SN_P,H_CALL_BUTTON_U_P,H_BATH_HELP_A_P,H_BATH_HELP_SN_P,H_BATH_HELP_U_P,H_COMP_5_A_P,H_COMP_5_SN_P,H_COMP_5_U_P,H_COMP_5_LINEAR_SCORE,H_COMP_5_STAR_RATING,H_MED_FOR_A_P,H_MED_FOR_SN_P,H_MED_FOR_U_P,H_SIDE_EFFECTS_A_P,H_SIDE_EFFECTS_SN_P,H_SIDE_EFFECTS_U_P,H_COMP_6_N_P,H_COMP_6_Y_P,H_COMP_6_LINEAR_SCORE,H_COMP_6_STAR_RATING,H_DISCH_HELP_N_P,H_DISCH_HELP_Y_P,H_SYMPTOMS_N_P,H_SYMPTOMS_Y_P,H_COMP_7_A,H_COMP_7_D_SD,H_COMP_7_SA,H_COMP_7_LINEAR_SCORE,H_COMP_7_STAR_RATING,H_CT_PREFER_A,H_CT_PREFER_D_SD,H_CT_PREFER_SA,H_CT_UNDER_A,H_CT_UNDER_D_SD,H_CT_UNDER_SA,H_CT_MED_A,H_CT_MED_D_SD,H_CT_MED_SA,H_CLEAN_HSP_A_P,H_CLEAN_HSP_SN_P,H_CLEAN_HSP_U_P,H_CLEAN_LINEAR_SCORE,H_CLEAN_STAR_RATING,H_QUIET_HSP_A_P,H_QUIET_HSP_SN_P,H_QUIET_HSP_U_P,H_QUIET_LINEAR_SCORE,H_QUIET_STAR_RATING,H_HSP_RATING_0_6,H_HSP_RATING_7_8,H_HSP_RATING_9_10,H_HSP_RATING_LINEAR_SCORE,H_HSP_RATING_STAR_RATING,H_RECMND_DN,H_RECMND_DY,H_RECMND_PY,H_RECMND_LINEAR_SCORE,H_RECMND_STAR_RATING,H_STAR_RATING,O_COMP_1_LINEAR_SCORE,O_COMP_2_LINEAR_SCORE,O_COMP_3_LINEAR_SCORE,O_PATIENT_RATE_LINEAR_SCORE,O_PATIENT_REC_LINEAR_SCORE,EDAC_30_AMI,EDAC_30_AMI_DEN,EDAC_30_HF,EDAC_30_HF_DEN,EDAC_30_PN,EDAC_30_PN_DEN,OP_32,OP_32_DEN,OP_35_ADM,OP_35_ADM_DEN,OP_35_ED,OP_35_ED_DEN,OP_36,OP_36_DEN,READM_30_CABG,READM_30_CABG_DEN,READM_30_COPD,READM_30_COPD_DEN,READM_30_HIP_KNEE,READM_30_HIP_KNEE_DEN,Hybrid_HWR,Hybrid_HWR_DEN,HAI_1_DEN_PRED,HAI_1_DEN_VOL,HAI_1,HAI_2_DEN_PRED,HAI_2_DEN_VOL,HAI_2,HAI_3_DEN_PRED,HAI_3_DEN_VOL,HAI_3,HAI_4_DEN_PRED,HAI_4_DEN_VOL,HAI_4,HAI_5_DEN_PRED,HAI_5_DEN_VOL,HAI_5,HAI_6_DEN_PRED,HAI_6_DEN_VOL,HAI_6,OP_13,OP_8,OP_10


In [28]:
measures = [
    'Hybrid_HWR',
      'Hybrid_HWM',
      'O_COMP_1_LINEAR_SCORE',
      'O_COMP_2_LINEAR_SCORE',
      'O_COMP_3_LINEAR_SCORE',
      'O_PATIENT_RATE_LINEAR_SCORE',
      'O_PATIENT_REC_LINEAR_SCORE',
      'H_COMP_1_LINEAR_SCORE',
      'H_COMP_2_LINEAR_SCORE', 
      'H_COMP_3_LINEAR_SCORE', 
      'H_COMP_5_LINEAR_SCORE', 
      'H_COMP_6_LINEAR_SCORE', 
      'H_COMP_7_LINEAR_SCORE', 
      'H_CLEAN_LINEAR_SCORE',  
      'H_QUIET_LINEAR_SCORE', 
      'H_RECMND_LINEAR_SCORE', 
      'H_HSP_RATING_LINEAR_SCORE',
]

for m in measures:
    ls = raw_data[m]
    print(m)
    print('   ', np.nanmin(ls), np.nanmin(ls), np.nanmedian(ls), np.nanmean(ls))

Hybrid_HWR
    0.1169999999999999 0.1169999999999999 0.149 0.14981497275527125
Hybrid_HWM
    0.018 0.018 0.042 0.04219096612296109
O_COMP_1_LINEAR_SCORE
    80.0 80.0 98.0 98.13780918727915
O_COMP_2_LINEAR_SCORE
    82.0 82.0 95.0 95.1387728878895
O_COMP_3_LINEAR_SCORE
    88.0 88.0 98.0 97.42274333440412
O_PATIENT_RATE_LINEAR_SCORE
    70.0 70.0 94.0 93.92579505300354
O_PATIENT_REC_LINEAR_SCORE
    68.0 68.0 93.0 92.57436556376486
H_COMP_1_LINEAR_SCORE
    78.0 78.0 91.0 91.2094125078964
H_COMP_2_LINEAR_SCORE
    76.0 76.0 91.0 90.74636765634871
H_COMP_3_LINEAR_SCORE
    62.0 62.0 83.0 83.53442830069488
H_COMP_5_LINEAR_SCORE
    53.0 53.0 76.0 76.23278584965256
H_COMP_6_LINEAR_SCORE
    66.0 66.0 87.0 86.35786481364498
H_COMP_7_LINEAR_SCORE
    67.0 67.0 81.0 80.89861023373342
H_CLEAN_LINEAR_SCORE
    69.0 69.0 87.0 86.70025268477575
H_QUIET_LINEAR_SCORE
    56.0 56.0 83.0 82.10518003790271
H_RECMND_LINEAR_SCORE
    64.0 64.0 87.0 86.96462413139608
H_HSP_RATING_LINEAR_SCORE
    70.0 